# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [42]:
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv


#### Creating list of filepaths to process original event csv data files

In [43]:
    """
In order to find filepath:
    checked current working directory
    got my current folder and subfolder event data
    created a for loop to create a list of files and collected each filepath
    joined the file path and roots with the subdirectories using glob
    printed file path list

    """
print(os.getcwd())
filepath = os.getcwd() + '/event_data'
for root, dirs, files in os.walk(filepath):
        file_path_list = glob.glob(os.path.join(root,'*'))

/home/workspace


#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [44]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
#print(len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length',\
                'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


In [45]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

## Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

## Begin writing your Apache Cassandra code in the cells below

#### Creating a Cluster

In [46]:
# (127.0.0.1)
"""
to connection to Cassandra I used cassandra.cluster
for executing queries I need a session

"""
from cassandra.cluster import Cluster
cluster = Cluster(['127.0.0.1'])

session = cluster.connect()

#### Create Keyspace

In [47]:
"""
I used try and except for creating keyspace and set class and replocation_factor.

"""
try:
    session.execute("""
    CREATE KEYSPACE IF NOT EXISTS udacity 
    WITH REPLICATION =
    {'class':'SimpleStrategy','replication_factor':1}""")
except Exception as e:
    print(e)

#### Set Keyspace

In [48]:
"""
Set Keyspace above created as keyspace
"""
try:
    session.set_keyspace('udacity')
except Exception as e:
    print(e)

### Now we need to create tables to run the following queries. Remember, with Apache Cassandra you model the database tables on the queries you want to run.

## Create queries to ask the following three questions of the data

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = 338, and itemInSession  = 4


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = 10, sessionid = 182
    

### 3. Give me every user name (first and last) in my music app history who listened to the song 'All Hands Against His Own'




In [49]:
"""
QUERY 1: I need to have artist, song title and song's length in the music app history that was heard during\
sessionId=338 and itemInSession=4. For that:
            I created song_info_by_session table and added certain column to the table with suitable column values. \
            As Primary Key I set unique values (sessionId,itemInSession). 
            Below I wrote query that gives exactly what I need.
            
"""
## sessionId = 338, and itemInSession = 4
query="CREATE TABLE IF NOT EXISTS song_info_by_session"
query=query+"(artist TEXT, first_name TEXT, itemInSession INT, length FLOAT, sessionId INT, song TEXT, PRIMARY KEY(itemInSession,sessionId))"

try:
    session.execute(query)
except Exception as e:
    print(e)

In [50]:
query="SELECT artist, length, song FROM song_info_by_session WHERE sessionId=338 AND itemInSession=4;"

try:
    session.execute(query)
except Exception as e:
    print(e)  


In [61]:
"""
We had csv file which I want to pull data from and insert it to my new table. I opened the file first and\
    wrote query to get right data to the table.

"""

file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:
        
        query = "INSERT INTO song_info_by_session(itemInSession,sessionId, artist, first_name)"
        query = query + "VALUES (%s, %s, %s, %s)"
        
        
        session.execute(query, (int(line[3]), int(line[8]), line[0], line[1]))
        
        

#### Do a SELECT to verify that the data have been inserted into each table

In [12]:
##to verify the data was entered into the table
query="SELECT * FROM song_info_by_session;"
try:
    session.execute(query)
except Exception as e:
    print(e)

### COPY AND REPEAT THE ABOVE THREE CELLS FOR EACH OF THE THREE QUESTIONS

In [62]:
"""
QUERY 2:
    I will use the same logic for the next two queries. For this query I pulled artist, song and user for userid=10 and sessionid=182
    As Primary key I used composite primary key. 
    Order by itemInSession
"""

## for userid = 10, sessionid = 182
query="CREATE TABLE IF NOT EXISTS song_info_by_user"
query=query+"(artist TEXT, first_name TEXT,itemInSession INT, last_name TEXT,sessionId INT, song TEXT, userId INT, PRIMARY KEY((sessionId, userId),itemInSession));"
try:
    session.execute(query)
except Exception as e:
    print(e)
                   

In [63]:
query="SELECT artist, first_name, last_name FROM song_info_by_user WHERE userId=10 AND sessionId=182 ORDER BY itemInSession;"

try:
    session.execute(query)
except Exception as e:
    print(e)  


In [64]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_info_by_user (sessionId, userId, itemInSession, artist, first_name, last_name, song)"
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
              
        session.execute(query, (int(line[8]), int(line[10]),int(line[3]),line[0],line[1], line[4],line[9]))
        

In [65]:
"""
QUERY 3: user name (first and last) in my music app history who listened to the song 'All Hands Against His Own
"""

query="CREATE TABLE IF NOT EXISTS song_info_by_song"
query=query+"(first_name text, last_name text, sessionId int, song text, userId int, PRIMARY KEY (song, userId))"
try:
    session.execute(query)
except Exception as e:
    print(e)

                    

In [66]:
query="""SELECT first_name, last_name FROM song_info_by_song WHERE song='All Hands Against His Own';"""

try:
    session.execute(query)
except Exception as e:
    print(e)  


In [17]:
file = 'event_datafile_new.csv'

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in csvreader:

        query = "INSERT INTO song_info_by_song (sessionId, userId, first_name, last_name, song)"
        query = query + "VALUES (%s, %s, %s, %s,%s)"
              
        session.execute(query, (int(line[8]), int(line[10]), line[1], line[4],line[10])))
        

### Drop the tables before closing out the sessions

In [131]:
try:
    session.execute("DROP TABLE song_info_by_session")
    session.execute("DROP TABLE song_info_by_user")
    session.execute("DROP TABLE song_info_by_song")
except Exception as e:
    print(e)

### Close the session and cluster connection¶

In [132]:
session.shutdown()
cluster.shutdown()